<a href="https://colab.research.google.com/github/world970511/RIDIBOOKS_romance_webnovel_review_Sentiment_Analysis/blob/main/anlay_ridi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import urllib.request
from sklearn.model_selection import train_test_split


urllib.request.urlretrieve("https://raw.githubusercontent.com/world970511/RIDIBOOKS_romance_webnovel_review_Sentiment_Analysis/main/data/result_all_data.txt",
                        filename="result_all_data.txt")
all_data = pd.read_csv('result_all_data.txt',encoding='utf-8')
print(all_data)

print('\n===================\n')
print('초반 데이터 확인 :',len(all_data))
all_data.drop_duplicates(subset=['review'], inplace=True)#중복된 리뷰들을 제거한다
print('중복 제거 확인 :',len(all_data))

all_data = all_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print('null값이 존재하는가?=',all_data.isnull().values.any()) # Null 값이 존재하는지 확인
print('\n===================\n')

train_data, test_data = train_test_split(all_data, test_size = 0.25, random_state = 42)#3:1로 데이터 분할
print('훈련용 리뷰의 개수 :', len(train_data))
print('테스트용 리뷰의 개수 :', len(test_data))

print('\n===================\n')
train_data.groupby('rate').size().reset_index(name = 'count')
test_data.groupby('rate').size().reset_index(name = 'count')

train_data=train_data.fillna(" ")#훈련 데이터에서 한글과 공백을 제외하고 제거
train_data['review']=train_data['review'].apply(lambda x:re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",x))
train_data['review']=train_data['review'].apply(lambda x:re.sub('^ +',"",x))
train_data['review'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how='any')
print('전처리 후 훈련용 샘플의 개수 :',len(train_data))

test_data=test_data.fillna(" ")#테스트 데이터에서 한글과 공백을 제외하고 제거
test_data['review']=test_data['review'].apply(lambda x:re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",x))
test_data['review']=test_data['review'].apply(lambda x:re.sub('^ +',"",x))
test_data['review'].replace('', np.nan, inplace=True)
test_data = test_data.dropna(how='any')
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))
print('\n===================\n')

test_data.to_csv('test.txt',mode='w',index=False)
train_data.to_csv('train.txt',mode='w',index=False)




       rate                                             review
0         1                                        사스가 내 취향...
1         1  재밌어요~ 근데 재탕은 조금 힘드네요.. 처음 읽을 때도 중간에 늘어지는 부분 있는...
2         1                                   시간 가는 줄 모르고 읽었네요
3         0  내내 3인칭이었다가 갑자가 1인칭 됐다가 ㅜㅜ  갑자기? 이런 순간들이 넘 많네요 ...
4         0  소개글에 속지 마세요. 걸크러시의 크도 없는 책입니다. 전통적인 문란쓰레기남x순진녀...
...     ...                                                ...
97561     0                         아..............제 취향은 아닌가봐요
97562     1                                     편한하게 웃으면서 읽었어요
97563     0                               읽다 포기... 기대를 너무 했나 봄
97564     1  오랜만에 몰입하며 읽었어요. 장면장면이 현실감 있게 쓰여져서 정말 드라마 한편을 보...
97565     1  한편의   대하드라마를 보는 느낌이었어요  은봄과  진언의  삶과  사랑  감동이었...

[97566 rows x 2 columns]


초반 데이터 확인 : 97566
중복 제거 확인 : 97566
null값이 존재하는가?= False


훈련용 리뷰의 개수 : 73174
테스트용 리뷰의 개수 : 24392


전처리 후 훈련용 샘플의 개수 : 73111
전처리 후 테스트용 샘플의 개수 : 24371




In [5]:
pip install konlpy

     |████████████████████████████████| 19.4MB 7.9MB/s 
     |████████████████████████████████| 460kB 38.5MB/s 
     |████████████████████████████████| 92kB 13.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [6]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))


train_data=pd.read_csv('train.txt',encoding='utf-8')
test_data=pd.read_csv('test.txt',encoding='utf-8')

okt = Okt()
stopwords = ['의','가','이','은','들','는','좀','원','권','과','도','를','으로','자','에','와','한','하다','뽕빨','제','금']

X_train = []
for sentence in train_data['review']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

X_test = []
for sentence in test_data['review']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

print('\n===================\n')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

y_train = np.array(train_data['rate'])
y_test = np.array(test_data['rate'])

drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))

print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))

max_len = 70
below_threshold_len(max_len, X_train)

X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)



단어 집합(vocabulary)의 크기 : 28561
등장 빈도가 2번 이하인 희귀 단어의 수: 15790
단어 집합에서 희귀 단어의 비율: 55.28517909036799
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.5018183364057616
단어 집합의 크기 : 12772


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


73089
리뷰의 최대 길이 : 722
리뷰의 평균 길이 : 17.435386993938895
전체 샘플 중 길이가 70 이하인 샘플의 비율: 97.15825910875782


In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

Epoch 1/15
975/975 [==============================] - 21s 13ms/step - loss: 0.3024 - acc: 0.8758 - val_loss: 0.2633 - val_acc: 0.8942

Epoch 00001: val_acc improved from -inf to 0.89417, saving model to best_model.h5
Epoch 2/15
975/975 [==============================] - 12s 12ms/step - loss: 0.2403 - acc: 0.9064 - val_loss: 0.2356 - val_acc: 0.9071

Epoch 00002: val_acc improved from 0.89417 to 0.90710, saving model to best_model.h5
Epoch 3/15
975/975 [==============================] - 12s 12ms/step - loss: 0.2178 - acc: 0.9175 - val_loss: 0.2357 - val_acc: 0.9082

Epoch 00003: val_acc improved from 0.90710 to 0.90820, saving model to best_model.h5
Epoch 4/15
975/975 [==============================] - 12s 12ms/step - loss: 0.2012 - acc: 0.9247 - val_loss: 0.2265 - val_acc: 0.9142

Epoch 00004: val_acc improved from 0.90820 to 0.91422, saving model to best_model.h5
Epoch 5/15
975/975 [==============================] - 12s 12ms/step - loss: 0.1889 - acc: 0.9307 - val_loss: 0.2350 - val_a